In [100]:
%load_ext autoreload
%autoreload 2
import torch

from sae.sparse_autoencoder import load_saved_sae
from sae.metrics import model_store_from_sae
from unlearning.metrics import get_output_probs_abcd
from unlearning.tool import UnlearningConfig, SAEUnlearningTool, MCQ_ActivationStoreAnalysis

from huggingface_hub import hf_hub_download
from datasets import load_dataset
import numpy as np

from jaxtyping import Float
from torch import Tensor

import plotly.express as px


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# resid pre 9
REPO_ID = "eoinf/unlearning_saes"
FILENAME = "jolly-dream-40/sparse_autoencoder_gemma-2b-it_blocks.9.hook_resid_pre_s16384_127995904.pt"


filename = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
sae = load_saved_sae(filename)

model = model_store_from_sae(sae)

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b-it into HookedTransformer
Moving model to device:  cuda


In [5]:
dataset_name = 'college_biology'

dataset = load_dataset("cais/mmlu", dataset_name, split='test')

answers = [x['answer'] for x in dataset]
questions = [x['question'] for x in dataset]
choices_list = [x['choices'] for x in dataset]


In [6]:
print(len(answers))

144


In [12]:

from unlearning.metrics import get_output_probs_abcd, convert_wmdp_data_to_prompt


In [13]:
tmp = convert_wmdp_data_to_prompt(question=questions[0], choices=choices_list[0], without_question=True)
print(tmp)

A. maintain the population at a point corresponding to the midpoint of its logistic curve
B. opt for zero population control once the K value of the curve has been reached
C. reduce the carrying capacity cif the environment to lower the K value
D. increase the mortality rate
Answer:


In [14]:
tmp

'A. maintain the population at a point corresponding to the midpoint of its logistic curve\nB. opt for zero population control once the K value of the curve has been reached\nC. reduce the carrying capacity cif the environment to lower the K value\nD. increase the mortality rate\nAnswer:'

In [29]:
from unlearning.metrics import all_permutations, calculate_MCQ_metrics

In [22]:
metrics = calculate_MCQ_metrics(model, dataset, permutations=all_permutations)

100%|██████████| 576/576 [02:05<00:00,  4.60it/s]


In [23]:
metrics['is_correct']

array([1., 1., 1., ..., 1., 0., 1.], dtype=float32)

In [24]:
metrics['mean_correct']

0.42476850748062134

In [25]:
(metrics['is_correct'].reshape(-1, 24).mean(axis=1) == 1)

array([0.9583333 , 0.08333334, 1.        , 0.875     , 1.        ,
       0.7083333 , 0.8333333 , 0.375     , 0.29166666, 1.        ,
       0.        , 0.        , 0.5416667 , 1.        , 0.25      ,
       0.29166666, 0.25      , 1.        , 0.        , 0.9583333 ,
       0.8333333 , 0.        , 0.7916667 , 1.        , 1.        ,
       1.        , 0.75      , 0.33333334, 0.29166666, 0.        ,
       1.        , 0.41666666, 0.5416667 , 0.20833333, 0.20833333,
       0.5       , 0.25      , 0.08333334, 0.9166667 , 0.08333334,
       0.25      , 0.75      , 1.        , 0.45833334, 0.16666667,
       0.125     , 0.25      , 0.5       , 0.25      , 0.        ,
       0.375     , 0.33333334, 0.20833333, 0.29166666, 0.08333334,
       0.5       , 0.5416667 , 0.25      , 0.        , 1.        ,
       0.        , 0.5416667 , 0.33333334, 0.375     , 0.7083333 ,
       0.25      , 0.        , 0.        , 1.        , 0.20833333,
       0.        , 0.625     , 0.9583333 , 0.        , 0.25   

In [26]:
(metrics['is_correct'].reshape(-1, 24).mean(axis=1) == 1).sum()

17

In [30]:
metrics_wo_question = calculate_MCQ_metrics(model, dataset, permutations=all_permutations, without_question=True)

100%|██████████| 576/576 [01:25<00:00,  6.76it/s]


In [34]:
metrics_wo_question['is_correct'].reshape(-1, 24).mean(axis=1)

array([1.        , 0.08333334, 0.5       , 0.5833333 , 0.7916667 ,
       0.7083333 , 0.29166666, 0.        , 0.        , 0.6666667 ,
       0.        , 0.25      , 0.5416667 , 0.9583333 , 0.29166666,
       0.125     , 0.08333334, 0.875     , 0.04166667, 0.29166666,
       0.9166667 , 0.        , 0.33333334, 0.625     , 0.45833334,
       0.29166666, 0.875     , 0.125     , 0.5833333 , 0.        ,
       0.        , 0.125     , 0.33333334, 0.625     , 0.04166667,
       0.7083333 , 0.25      , 0.125     , 0.33333334, 0.        ,
       0.20833333, 0.5       , 1.        , 0.04166667, 0.        ,
       0.6666667 , 0.20833333, 0.29166666, 0.20833333, 0.        ,
       0.5       , 0.        , 0.        , 0.20833333, 0.125     ,
       0.5       , 0.33333334, 0.45833334, 0.16666667, 0.20833333,
       0.25      , 0.25      , 0.16666667, 0.375     , 0.5833333 ,
       0.25      , 0.04166667, 0.04166667, 0.25      , 0.6666667 ,
       0.16666667, 0.375     , 0.04166667, 0.        , 0.33333

In [74]:
import os

def save_target_question_ids(model, dataset, output_dir='../data/'):
    """
    Find and save the question ids where the model 
    1. get all permutations correct
    2. get all permutations correct but not all permutations correct when without the instruction and question prompt
    """
    metrics = calculate_MCQ_metrics(model, dataset, permutations=all_permutations)
    metrics_wo_question = calculate_MCQ_metrics(model, dataset, permutations=all_permutations, without_question=True)
    
    each_question_acc = metrics['is_correct'].reshape(-1, 24)
    each_question_acc_wo_question = metrics_wo_question['is_correct'].reshape(-1, 24)
    
    # find the question ids where accuracy is 1 in each_question_acc but not 1 in
    # each_question_acc_wo_question
    questions_correct = each_question_acc.mean(axis=1) == 1
    questions_correct_but_not_wo_question = questions_correct & (each_question_acc_wo_question.mean(axis=1) != 1)
    
    correct_question_id = np.where(questions_correct)[0]
    questions_correct_but_not_wo_question_id = np.where(questions_correct_but_not_wo_question)[0]
    
    # save to file
    file_name_prefix = f'{dataset._info.dataset_name}_{dataset.config_name}_{model.cfg.model_name}'
    file_name_prefix = os.path.join(output_dir, file_name_prefix)
    
    np.savetxt(f"{file_name_prefix}_correct.csv", correct_question_id)
    np.savetxt(f"{file_name_prefix}_correct_not_correct_wo_question_prompt.csv", questions_correct_but_not_wo_question_id)
    
    print("Found correct questions: ", len(correct_question_id))
    print("Found correct questions but not correct without question prompt: ", len(questions_correct_but_not_wo_question_id))
    
    
    
    

In [73]:
np.savetxt(f"../data/tmp.csv", np.array([1,2,3]))

In [70]:
import sys

sys.path

['/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/root/unlearning/.venv/lib/python3.10/site-packages',
 '/root/unlearning',
 '/tmp/tmp_lrl6yjc']

In [67]:
file_name_prefix = f'{dataset._info.dataset_name}_{dataset.config_name}_{model.cfg.model_name}'
file_name_prefix = os.path.join('./data/', file_name_prefix)

file_name_prefix

'./data/mmlu_college_biology_gemma-2b-it'

In [75]:
save_target_question_ids(model, dataset)

100%|██████████| 576/576 [01:27<00:00,  6.59it/s]

Found correct questions:  17
Found correct questions but not correct without question prompt:  16


In [41]:
# get the question ids
question_ids = np.where(tmp)[0]

In [51]:
dataset.__dir__()

['_info',
 '_split',
 '_indexes',
 '_data',
 '_indices',
 '_format_type',
 '_format_kwargs',
 '_format_columns',
 '_output_all_columns',
 '_fingerprint',
 '__module__',
 '__doc__',
 '__init__',
 'features',
 'from_file',
 'from_buffer',
 'from_pandas',
 'from_polars',
 'from_dict',
 'from_list',
 'from_csv',
 'from_generator',
 'from_json',
 'from_parquet',
 'from_text',
 'from_spark',
 'from_sql',
 '__setstate__',
 '__del__',
 '__enter__',
 '__exit__',
 'save_to_disk',
 '_save_to_disk_single',
 '_build_local_temp_path',
 'load_from_disk',
 'data',
 'cache_files',
 'num_columns',
 'num_rows',
 'column_names',
 'shape',
 'unique',
 'class_encode_column',
 'flatten',
 'cast',
 'cast_column',
 'remove_columns',
 'rename_column',
 'rename_columns',
 'select_columns',
 '__len__',
 '__iter__',
 'iter',
 '__repr__',
 'format',
 'formatted_as',
 'set_format',
 'reset_format',
 'set_transform',
 'with_format',
 'with_transform',
 'prepare_for_task',
 '_getitem',
 '__getitem__',
 '__getitems__',

In [56]:
dataset._info

DatasetInfo(description='', citation='', homepage='', license='', features={'question': Value(dtype='string', id=None), 'subject': Value(dtype='string', id=None), 'choices': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'answer': ClassLabel(names=['A', 'B', 'C', 'D'], id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name='parquet', dataset_name='mmlu', config_name='college_biology', version=0.0.0, splits={'test': SplitInfo(name='test', num_bytes=51539, num_examples=144, shard_lengths=None, dataset_name='mmlu'), 'validation': SplitInfo(name='validation', num_bytes=5113, num_examples=16, shard_lengths=None, dataset_name='mmlu'), 'dev': SplitInfo(name='dev', num_bytes=1616, num_examples=5, shard_lengths=None, dataset_name='mmlu')}, download_checksums={'hf://datasets/cais/mmlu@c30699e8356da336a370243923dbaf21066bb9fe/college_biology/test-00000-of-00001.parquet': {'num_bytes': 31845, 'checksum': None}, 'hf://datasets/cais/mmlu@c30699

In [59]:
model.cfg.model_name

'gemma-2b-it'

In [78]:
np.where(metrics_wo_question['is_correct'].reshape(-1, 24).mean(axis=1) == 1)[0]

array([ 0, 42])

#### Add new dataset

In [91]:
for dataset_name in ['high_school_us_history']: # ['abstract_algebra', 'college_computer_science', 'high_school_geography', 'human_aging']:

    dataset = load_dataset("cais/mmlu", dataset_name, split='test')
    save_target_question_ids(model, dataset)

100%|██████████| 816/816 [02:16<00:00,  5.99it/s]

Found correct questions:  28
Found correct questions but not correct without question prompt:  27


In [101]:

filename = "../data/wmdp-bio_gemma_2b_it_correct.csv"
correct_question_ids = np.genfromtxt(filename)

dataset_args = {
    'question_subset': correct_question_ids,
}

sae.cfg.n_batches_in_store_buffer = 86
# unlearn_activation_store = MCQ_ActivationStoreAnalysis(sae.cfg, model, dataset_args=dataset_args)
unlearning_metric = 'wmdp-bio_gemma_2b_it_correct'
control_metric = ['abstract_algebra', 'college_computer_science', 'high_school_geography']

# unlearn_cfg = UnlearningConfig(unlearn_activation_store=None, unlearning_metric=unlearning_metric, control_metric=control_metric)
unlearn_cfg = UnlearningConfig(unlearn_activation_store=None, unlearning_metric=unlearning_metric)
ul_tool = SAEUnlearningTool(unlearn_cfg)
ul_tool.setup(create_base_act_store=False, create_unlearn_act_store=False, model=model)

In [102]:
unlearn_cfg.control_metric

['high_school_us_history',
 'college_computer_science',
 'high_school_geography',
 'human_aging',
 'college_biology']

In [103]:
features_to_ablate = [7983, 16186, 12273, 14315, 4342, 10051, 15858, 6958]
multiplier = 20

ablate_params = {
    'features_to_ablate': features_to_ablate,
    'multiplier': multiplier,
    'intervention_method': 'scale_feature_activation',
    'permutations': [[0,1,2,3]]
}

# metrics = ul_tool.calculate_metrics(**ablate_params)


# calc control metric and loss
control_metrics = ul_tool.calculate_control_metrics(**ablate_params)

# loss_added = ul_tool.compute_loss_added(n_batch=30, **ablate_params)

100%|██████████| 3/3 [00:01<00:00,  2.65it/s]


In [105]:
control_metrics

{'high_school_us_history': {'mean_correct': 0.48148149251937866,
  'total_correct': 13,
  'is_correct': array([0., 0., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1.,
         0., 0., 0., 0., 1., 0., 1., 0., 1., 1.], dtype=float32),
  'output_probs': array([[9.8362988e-01, 1.9729647e-04, 2.0095882e-04, 1.5598402e-02],
         [5.2172208e-04, 1.5751764e-04, 9.9605465e-01, 7.5944222e-06],
         [1.7719527e-01, 4.3885824e-03, 7.9074991e-01, 3.6680204e-04],
         [9.9194568e-01, 5.3297589e-07, 2.0780908e-07, 4.3941171e-08],
         [4.1224668e-03, 1.1418777e-04, 9.5532018e-01, 4.6363533e-05],
         [4.5608860e-01, 1.7948341e-02, 4.2773053e-01, 6.0959820e-02],
         [9.9215615e-01, 8.9372548e-07, 1.4806488e-06, 1.6940235e-03],
         [9.1611344e-01, 7.5462711e-05, 8.0279678e-02, 6.9963098e-06],
         [7.2069401e-05, 2.9160106e-03, 9.9315602e-01, 8.4834446e-06],
         [2.6987885e-07, 9.9895644e-01, 8.2351198e-06, 1.6978831e-07],
         [1.0930685e-04, 1.00